# Linear Regression

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import sys
from pathlib import Path

import pandas as pd

sys.path.append("../src")
sys.path.append("../scripts")

from model import RegimeSwitchingModel
from data import create_train_test_data

## Setting

In [3]:
# パス，変数
# ディレクトリ
# DATA_INPUT_DIR = Path(r"O:\プロジェクト\P00770\01_Simulation\Linear_Model") # インプットデータディレクトリ
DATA_INPUT_DIR = Path("../data/input")
DATA_OUTPUT_DIR = Path("../data/output") # 出力データディレクトリ
# 分析条件
prediction_lag = 1
corr_rolling_span = 250
rolling_span = corr_rolling_span
# ファイル
endog_file = DATA_INPUT_DIR / "X.csv"
exog_file = DATA_INPUT_DIR / "Y.csv"
selected_factors_file = DATA_INPUT_DIR / f"Selected_Factors_Col{corr_rolling_span}_lag{prediction_lag}.csv"
# 日付リスト作成
_df = pd.read_csv(selected_factors_file, index_col=0)
_df.index = pd.to_datetime(_df.index)
date_list = _df.index.to_list()[:-1]
del _df

## Check data

In [45]:
base_date: datetime.datetime = datetime.datetime(2014, 12, 11) # 基準日
# 説明変数を正規化するか
normalize_exog: bool = True
normalize_exog_type = "z2" # @params ["divmax", "z"]
# 非説明変数を正規化するか
normalize_endog: bool = False
normalize_endog_type = "z2" #@params ["divmax", "z"]


y_train, y_test, X_train, X_test = create_train_test_data(
    base_date,
    endog_file,
    exog_file,
    # selected_factors_csv_file = selected_factors_file,
    selected_factors_csv_file=None,
    prediction_lag = prediction_lag,
    rolling_span = rolling_span,
    normalize_exog = normalize_exog,
    normalize_exog_type = normalize_exog_type,
    varbose=False
)

# summarize
print(
    "summarize\n"
    f"base date : {base_date}\n"
    f"train X data: {X_train.index[0]} - {X_train.index[-1]}\n"
    f"train y data: {y_train.index[0]} - {y_train.index[-1]}\n"
)

summarize
base date : 2014-12-11 00:00:00
train X data: 2013-12-02 00:00:00 - 2014-12-09 00:00:00
train y data: 2013-12-03 00:00:00 - 2014-12-10 00:00:00



In [46]:
X_train

,US10YR_Diff,US2YR_Diff,US10_2_Diff,Ted_Spread_Diff,VIX_Diff,MOVE_Diff,Volatility_Index_Japan_Diff,OVX_Diff,Currency_Volatility_Diff,SP500_Return,...,Global_Financial_Stress_Index_Diff,US_Economic_Policy_Uncertainty_Index_Diff,WTI_Oil_Return,Natural_Gas_Return,Commodity_Index_Return,EURUSD_Return,USDJPY_Return,US_Dollar_Index_Return,United_Kingdom_Daily_News_Index_Diff,US_FRB_Daily_News_Economic_Sentiment_Diff
date,,,,,,,,,,,,,,,,,,,,,
2013-12-02,1.342797,0.123616,1.671942,-0.094848,0.487601,1.765621,0.515850,0.189343,0.584846,-0.486414,...,0.299916,0.512055,0.919574,0.194410,0.763887,-0.838862,0.555289,0.847203,0.020684,2.798453
2013-12-03,-0.266734,-0.056353,-0.309095,-2.225362,0.292650,-0.361551,0.097191,-0.776510,0.482529,-0.556940,...,1.313145,-0.300305,1.738354,-0.022662,1.294090,0.604040,-0.072685,-1.440423,-0.509609,-0.654322
2013-12-04,1.370811,0.123616,1.708690,0.926610,0.134832,1.266517,0.384050,-0.451550,1.249910,-0.275737,...,1.313145,-0.122278,0.934468,0.151284,0.501860,0.542969,-0.698215,-0.011541,1.592855,-1.012107
2013-12-05,1.009176,0.488169,0.970394,1.116737,0.348350,0.105595,1.244628,-0.487657,-0.847598,-0.727612,...,0.637659,-0.675046,0.224792,0.406190,-0.183830,0.262028,-1.185349,-1.643471,-0.904623,0.392402
2013-12-06,-0.363510,-0.051739,-0.439383,-1.348628,-1.201976,-2.333871,-0.290457,-0.641110,-1.308026,1.590888,...,-1.051056,-0.292224,0.288692,0.663115,0.453812,2.242371,1.697012,0.195904,1.698073,-0.099126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-03,-0.261641,0.857337,-0.963873,0.593486,-0.357188,-0.719279,1.120581,-2.238830,2.426560,0.478604,...,0.637659,0.204737,0.615091,-0.527687,-0.345462,-2.129372,0.791938,0.990594,-0.930691,0.209539
2014-12-04,-1.112247,-0.790075,-0.887037,1.437277,-0.087969,-0.387281,0.213485,-0.758457,-3.303216,-0.254648,...,-0.375570,-0.685904,-0.490774,-0.415387,-0.353331,-0.317167,0.839966,-1.027085,1.102504,-0.694499
2014-12-05,1.897984,4.724372,-0.930466,-0.275352,-0.524289,0.071357,-0.499786,-1.137577,0.584846,0.166105,...,0.299916,0.352714,-0.911265,-0.598756,-0.412418,-0.604986,2.513275,2.145441,-0.279833,0.710010


In [60]:
df_norm = pd.read_excel(
    "../data/input/normalization_check.xlsx",
    skiprows=9,
    index_col=0
).iloc[:, 0:33]